In [106]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [80]:
data = pd.read_csv('/Users/inigoparra/Desktop/carpeta sin título/clean-master.csv')

In [81]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

X_train = train_data['text']
y_train = train_data['label']

X_test = test_data['text']
y_test = test_data['label']

X_val = val_data['text']
y_val = val_data['label']

In [82]:
with open('/Users/inigoparra/Desktop/GitHub Repositories/WiBaSets/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
        
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    return ' '.join(filtered_tokens)

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)
X_val = X_val.apply(preprocess_text)

In [83]:
import re
import nltk

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[0-9]', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and emojis
    text = text.split()  # Tokenize the text
    text = ' '.join(text)
    return text

X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)
X_val = X_val.apply(clean_text)

print(X_train[1])

irailaren eaj bozkatzea izango herri gisa gizarte gisa aurrera egitea


In [84]:
max_features = 20500
max_len = 100
embedding_dim = 300

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

In [109]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=max_len))
model.add(Bidirectional(GRU(32, return_sequences=True,
               kernel_regularizer=regularizers.l2(0.02), 
               recurrent_regularizer=regularizers.l2(0.02))))
model.add(Dropout(0.6))
model.add(Bidirectional(GRU(16)))
model.add(Dropout(0.6))
model.add(Dense(3, activation='softmax'))

optimizer = Adam(lr=0.00005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [110]:
batch_size = 24
epochs = 10
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[early_stopping]
)

Epoch 1/10
293/293 [==============================] - 22s 65ms/step - loss: 7.1512 - accuracy: 0.3516 - val_loss: 6.2063 - val_accuracy: 0.3474
Epoch 2/10
293/293 [==============================] - 19s 63ms/step - loss: 5.4426 - accuracy: 0.3766 - val_loss: 4.7552 - val_accuracy: 0.3449
Epoch 3/10
293/293 [==============================] - 18s 63ms/step - loss: 4.1907 - accuracy: 0.3834 - val_loss: 3.6922 - val_accuracy: 0.3423
Epoch 4/10
293/293 [==============================] - 18s 62ms/step - loss: 3.2768 - accuracy: 0.3924 - val_loss: 2.9180 - val_accuracy: 0.3372
Epoch 5/10
293/293 [==============================] - 18s 63ms/step - loss: 2.6129 - accuracy: 0.4065 - val_loss: 2.3611 - val_accuracy: 0.3359
Epoch 6/10
293/293 [==============================] - 18s 63ms/step - loss: 2.1374 - accuracy: 0.4121 - val_loss: 1.9649 - val_accuracy: 0.3372
Epoch 7/10
293/293 [==============================] - 18s 63ms/step - loss: 1.8025 - accuracy: 0.4183 - val_loss: 1.6884 - val_accuracy:

In [117]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

39/39 [==============================] - 1s 21ms/step - loss: 1.1108 - accuracy: 0.3870
Test loss: 1.1107869148254395, Test accuracy: 0.38701921701431274
